In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [11]:
# Load libraries
import numpy as np
import pandas as pd
import sys
import os
import pickle

from classes import boundaries as bd
plot_setup = bd.plot_setup()

%config Completer.use_jedi = False 




#--- Notebook run settings
run = 'command line' # 'manual' 'command line'
#run = 'manual'


# Settings for the multiple linear regression model (MLR)
#--- Arguments
if run == 'command line':
    # Read command line arguments
    arguments = os.environ['NB_ARGS']
    alpha, date_exp_str = arguments.split(',')
    
    alpha              = float(alpha)   
    
elif run == 'manual':
    # Enter arguments manually 
    alpha              = 0.01
    date_exp_str       = '230207_Full'
    remove_data        = True

print(date_exp_str, alpha)    

230117_Management 0.01


In [12]:
import glob
print(glob.glob('site_reg_results/'+date_exp_str+'*_SITE_best_models_dAIC.pkl'))
print(glob.glob('spe_reg_results/'+date_exp_str+'*_SPE_best_models_dAIC.pkl'))

['site_reg_results/230117_Management_0.01_SITE_best_models_dAIC.pkl']
['spe_reg_results/230117_Management_0.01_SPE_best_models_dAIC.pkl']


In [13]:
#---------------#
#  Import data  #
#---------------#
mtf_type = ['count','cmass']
mortality_groups = ['M_All','M_Fire','M_NoFire','M_Other']

with open('site_reg_results/'+date_exp_str+'_'+str(alpha)+'_SITE_best_models_dAIC.pkl','rb') as f:
    df_res_mort_site = pickle.load(f)
    
with open('spe_reg_results/'+date_exp_str+'_'+str(alpha)+'_SPE_best_models_dAIC.pkl','rb') as f:
    df_res_mort_spe = pickle.load(f)    

In [14]:
# Table B2
bd.print_header('Table B2')
df_all_mort_levels = pd.concat([df_res_mort_site,df_res_mort_spe])
display(df_all_mort_levels)

with open(date_exp_str+'_TableB2_df.pkl','wb') as f:
    pickle.dump(df_all_mort_levels,f)


#--------------#
#   Table B2   #
#--------------#



r2          AIC  \
                                                        count cmass  count   
Level   Mortality model                                                      
Site    M_All     MTF ~ MAT + DBH_mean + Managed         0.36  0.42    147   
                  MTF ~ MATsoil + DBH_mean + Managed     0.34  0.41  148.7   
        M_Fire    MTF ~ DBH_mean                         0.55  0.48   36.5   
        M_NoFire  MTF ~ CDI + Managed                    0.35  0.38  107.3   
                  MTF ~ MAT                              0.32  0.38  107.8   
                  MTF ~ MATsoil                          0.32  0.38  107.8   
        M_Other   MTF ~ CDI + Managed                     NaN  0.53    NaN   
                  MTF ~ MATsoil                          0.37  0.46   85.9   
                  MTF ~ MAT                              0.37  0.43   86.3   
                  MTF ~ CDI                              0.38   NaN   85.2   
Species M_All     MTF ~ MAT + Humidity + PFT + DBH_mean  0.65   NaN  190.9   
                  MTF ~ MAT + Humidity + PFT + DBH_mean   NaN  0.63    NaN   
                  MTF ~ MATsoil + DBH_mean + Managed     0.62   NaN  197.6   
                  MTF ~ MAT + Humidity + DBH_mean         NaN   0.6    NaN   
        M_Fire    MTF ~ DBH_mean                         0.71  0.66   72.3   
                  MTF ~ MAT + Humidity + Managed          NaN  0.66    NaN   
        M_NoFire  MTF ~ MAT + PFT + DBH_mean             0.54  0.54  110.5   
                  MTF ~ MAT + DBH_mean                    0.5  0.51  116.2   
                  MTF ~ MAT + PFT                         0.5   NaN    116   
                  MTF ~ MATsoil + DBH_mean                NaN  0.49    NaN   
        M_Other   MTF ~ MAT + PFT + DBH_mean              NaN  0.59    NaN   
                  MTF ~ MAT + PFT + DBH_mean             0.58   NaN   94.8   
                  MTF ~ CDI + Managed                    0.57  0.55   95.9   
                  MTF ~ MAT + DBH_mean                    NaN  0.54    NaN   
                  MTF ~ CDI + PFT                        0.53   NaN  100.6   
                  MTF ~ MAT + PFT                        0.53   NaN  100.6   

                                                               d_AIC        \
                                                         cmass count cmass   
Level   Mortality model                                                      
Site    M_All     MTF ~ MAT + DBH_mean + Managed         150.8     0     0   
                  MTF ~ MATsoil + DBH_mean + Managed     152.2   1.7   1.4   
        M_Fire    MTF ~ DBH_mean                          39.4     0     0   
        M_NoFire  MTF ~ CDI + Managed                    113.7     0   0.9   
                  MTF ~ MAT                              113.3   0.5   0.5   
                  MTF ~ MATsoil                          112.8   0.5     0   
        M_Other   MTF ~ CDI + Managed                     84.1   NaN     0   
                  MTF ~ MATsoil                           88.1   0.7     4   
                  MTF ~ MAT                                 90   1.1   5.9   
                  MTF ~ CDI                                NaN     0   NaN   
Species M_All     MTF ~ MAT + Humidity + PFT + DBH_mean    NaN     0   NaN   
                  MTF ~ MAT + Humidity + PFT + DBH_mean  214.2   NaN     0   
                  MTF ~ MATsoil + DBH_mean + Managed       NaN   6.7   NaN   
                  MTF ~ MAT + Humidity + DBH_mean        220.8   NaN   6.6   
        M_Fire    MTF ~ DBH_mean                          82.7     0     0   
                  MTF ~ MAT + Humidity + Managed          83.9   NaN   1.2   
        M_NoFire  MTF ~ MAT + PFT + DBH_mean               123     0     0   
                  MTF ~ MAT + DBH_mean                   126.6   5.7   3.6   
                  MTF ~ MAT + PFT                          NaN   5.5   NaN   
                  MTF ~ MATsoil + DBH_mean               129.9   NaN   6.9   
  

In [15]:
# Table 4
bd.print_header('Table 4 - '+date_exp_str.split('_')[1])

#df_all_mort_levels = df_all_mort_levels.reset_index().set_index(['Level','Mortality','model','climate']).copy()

df_best_mort_model = pd.DataFrame()

for t,tpe in enumerate(mtf_type):
    df_mtf_type = df_all_mort_levels.copy().swaplevel(axis=1)[tpe].reset_index()
    
    # Select model by mrotality group with the minimum RMSE when ∆RMSE < 0.5 years
    df_mtf_type['RMSE_min_diff'] = (df_mtf_type.RMSE - 
                                    df_mtf_type.groupby(['Level','Mortality'])['RMSE'].transform('min'))    
    df_min_rmse = df_mtf_type[df_mtf_type['RMSE_min_diff'] < 0.5]
    df_min_rmse.reset_index(inplace=True)
    
    # Otherwise choose largest variance explained
    df_max_r2 = df_min_rmse.groupby(['Level','Mortality'])['r2'].max().to_frame()
    max_r2_idx = [np.where(df_min_rmse.r2 == i) for i in df_max_r2.r2.tolist()]
    max_r2_idx_flat = [item for sublist in max_r2_idx for item in sublist]
    max_r2_idx_flat = [item for sublist in max_r2_idx_flat for item in sublist]
    df_best_mort = df_min_rmse.loc[df_min_rmse.index.isin(max_r2_idx_flat),:]
    
    # Format dataframe
    df_best_mort.set_index(['Level','Mortality','model'], inplace=True)
    df_best_mort = df_best_mort.drop('index',axis=1).copy()
    df_best_mort.columns = pd.MultiIndex.from_tuples([(i,tpe) for i in df_best_mort.columns])
    df_best_mort_model = pd.concat([df_best_mort_model,df_best_mort],axis=1)

    
df_best_mort_model['N_obs']  = df_all_mort_levels.reset_index().groupby(['Level','Mortality'])['N_obs'].first()

# If there are <20 observations set results to NaN (Management exclusion, Site, Fire mortality)
#df_best_mort_model[df_best_mort_model.N_obs < 20] = np.nan

df_best_mort_model = df_best_mort_model.reset_index().set_index(['Level','Mortality','model'])
#df_best_mort_model['Climate'] = df_best_mort_model.climate 

# Store result
with open(date_exp_str+'_Table4_df.pkl','wb') as f:
    pickle.dump(df_best_mort_model[['r2','RMSE','ID','N_obs']],f)
display(df_best_mort_model[['r2','RMSE','N_obs']])
df_best_mort_model[['r2','RMSE','ID']]
print(df_best_mort_model[['r2','RMSE','ID']].to_latex())


#--------------------------#
#   Table 4 - Management   #
#--------------------------#



r2        RMSE  \
                                                        count cmass count   
Level   Mortality model                                                     
Site    M_All     MTF ~ MAT + DBH_mean + Managed         0.36  0.42  23.5   
        M_Fire    MTF ~ DBH_mean                         0.55  0.48  21.0   
        M_NoFire  MTF ~ MAT                              0.32  0.38  23.0   
        M_Other   MTF ~ MAT                              0.37  0.43  24.4   
Species M_All     MTF ~ MAT + Humidity + PFT + DBH_mean  0.65  0.63  25.7   
        M_Fire    MTF ~ DBH_mean                         0.71  0.66  38.3   
        M_NoFire  MTF ~ MAT + PFT + DBH_mean             0.54  0.54  12.4   
        M_Other   MTF ~ MAT + PFT + DBH_mean             0.58  0.59  12.6   

                                                               N_obs  
                                                        cmass         
Level   Mortality model                                               
Site    M_All     MTF ~ MAT + DBH_mean + Managed         49.3   64.0  
        M_Fire    MTF ~ DBH_mean                         54.9   17.0  
        M_NoFire  MTF ~ MAT                              45.0   47.0  
        M_Other   MTF ~ MAT                              47.8   37.0  
Species M_All     MTF ~ MAT + Humidity + PFT + DBH_mean  43.7  107.0  
        M_Fire    MTF ~ DBH_mean                         67.8   37.0  
        M_NoFire  MTF ~ MAT + PFT + DBH_mean             18.9   70.0  
        M_Other   MTF ~ MAT + PFT + DBH_mean             19.6   61.0

\begin{tabular}{lllllrrll}
\toprule
        &         &                            & \multicolumn{2}{l}{r2} & \multicolumn{2}{l}{RMSE} & \multicolumn{2}{l}{ID} \\
        &         &                            & count & cmass & count & cmass & count & cmass \\
Level & Mortality & model &       &       &       &       &       &       \\
\midrule
Site & M\_All & MTF \textasciitilde  MAT + DBH\_mean + Managed &  0.36 &  0.42 &  23.5 &  49.3 &     1 &     0 \\
        & M\_Fire & MTF \textasciitilde  DBH\_mean &  0.55 &  0.48 &  21.0 &  54.9 &     0 &     0 \\
        & M\_NoFire & MTF \textasciitilde  MAT &  0.32 &  0.38 &  23.0 &  45.0 &    28 &    26 \\
        & M\_Other & MTF \textasciitilde  MAT &  0.37 &  0.43 &  24.4 &  47.8 &    33 &    34 \\
Species & M\_All & MTF \textasciitilde  MAT + Humidity + PFT + DBH\_mean &  0.65 &  0.63 &  25.7 &  43.7 &     1 &     1 \\
        & M\_Fire & MTF \textasciitilde  DBH\_mean &  0.71 &  0.66 &  38.3 &  67.8 &     1 &     0 \\
        & M\_NoF